In [6]:
%pip install voila

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import folium
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
from shapely.geometry import box
from geopy.geocoders import Nominatim
from IPython.display import display, clear_output
from ipywidgets import Text, HTML, VBox, Output, Dropdown, Button
from osmnx.distance import euclidean

# --- Configuration ---
north, south, east, west = 19.2, 18.9, 72.95, 72.75
bbox_polygon = box(west, south, east, north)
graph_file = "mumbai_drive.graphml"

# --- Initialize ---
geolocator = Nominatim(user_agent="route_planner")
clicked_points = {"start": None, "end": None}
output_map, output_distance, output_elevation = Output(), Output(), Output()

# --- Load or Download Graph ---
if os.path.exists(graph_file):
    graph = ox.load_graphml(graph_file)
else:
    graph = ox.graph_from_polygon(bbox_polygon, network_type='drive')
    ox.save_graphml(graph, graph_file)

# --- Simulated Blocked Roads ---
def get_edge_weight(u, v, k):
    data = graph[u][v][k]
    if 'name' in data and isinstance(data['name'], str) and 'blocked' in data['name'].lower():
        return 1e6
    return data.get('length', 1e5)

# --- Speed & Color Mappings ---
road_speeds = {
    'motorway': 60, 'trunk': 50, 'primary': 40, 'secondary': 35,
    'tertiary': 30, 'residential': 25, 'unclassified': 20
}
road_colors = {
    'motorway': 'red', 'trunk': 'darkred', 'primary': 'orange',
    'secondary': 'yellow', 'tertiary': 'lightgreen',
    'residential': 'green', 'unclassified': 'gray'
}

def get_edge_color(u, v, k):
    highway = graph[u][v][k].get("highway", "unclassified")
    if isinstance(highway, list):
        highway = highway[0]
    return road_colors.get(highway, 'gray')

def get_edge_speed(u, v, k):
    highway = graph[u][v][k].get("highway", "unclassified")
    if isinstance(highway, list):
        highway = highway[0]
    return road_speeds.get(highway, 20)

# --- A* Heuristic ---
def heuristic(n1, n2):
    x1, y1 = graph.nodes[n1]['x'], graph.nodes[n1]['y']
    x2, y2 = graph.nodes[n2]['x'], graph.nodes[n2]['y']
    return ((x1 - x2)**2 + (y1 - y2)**2)**0.5

def get_coordinates(place):
    location = geolocator.geocode(place)
    return (location.latitude, location.longitude) if location else (None, None)

def plot_elevation(route):
    elevations = [graph.nodes[n].get("elevation", 0) for n in route]
    distances = [0]
    for i in range(1, len(route)):
        u, v = route[i-1], route[i]
        dist = euclidean(graph.nodes[u]['y'], graph.nodes[u]['x'], graph.nodes[v]['y'], graph.nodes[v]['x'])
        distances.append(distances[-1] + dist)
    plt.figure(figsize=(8, 3))
    plt.plot(distances, elevations, color='purple')
    plt.xlabel("Distance (m)")
    plt.ylabel("Elevation (m)")
    plt.title("Elevation Profile")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def add_pois_to_map(m, lat, lon):
    tags = {"amenity": True, "tourism": True, "shop": True}
    try:
        pois = ox.features_from_point((lat, lon), tags=tags, dist=1000)
    except AttributeError:
        from osmnx.features import features_from_point
        pois = features_from_point((lat, lon), tags=tags, dist=1000)

    for _, row in pois.iterrows():
        if 'name' in row and row.geometry.geom_type == 'Point':
            folium.Marker(
                location=[row.geometry.y, row.geometry.x],
                popup=row['name'],
                icon=folium.Icon(color='blue', icon='info-sign')
            ).add_to(m)

def calculate_route(start, end, mode='shortest', use_click=False):
    clear_output(wait=True)

    if use_click:
        start_lat, start_lon = clicked_points['start']
        end_lat, end_lon = clicked_points['end']
    else:
        start_lat, start_lon = get_coordinates(start)
        end_lat, end_lon = get_coordinates(end)

    if None in (start_lat, start_lon, end_lat, end_lon):
        print("Invalid locations.")
        return

    try:
        start_node = ox.distance.nearest_nodes(graph, X=start_lon, Y=start_lat)
        end_node = ox.distance.nearest_nodes(graph, X=end_lon, Y=end_lat)
    except:
        print("Error finding route nodes.")
        return

    if not nx.has_path(graph, start_node, end_node):
        print("No path found.")
        return

    weight_func = lambda u, v, d: get_edge_weight(u, v, 0)
    route = nx.astar_path(graph, start_node, end_node, heuristic=heuristic, weight=weight_func)

    route_coords = [(graph.nodes[n]['y'], graph.nodes[n]['x']) for n in route]
    route_length = sum(get_edge_weight(u, v, 0) for u, v in zip(route[:-1], route[1:]))

    total_time = 0
    for u, v in zip(route[:-1], route[1:]):
        speed = get_edge_speed(u, v, 0)
        length = get_edge_weight(u, v, 0)
        total_time += (length / 1000) / speed * 60  # time in minutes

    m = folium.Map(location=route_coords[len(route_coords)//2], zoom_start=13, control_scale=True)
    folium.TileLayer('openstreetmap').add_to(m)
    folium.TileLayer('Stamen Terrain').add_to(m)
    folium.TileLayer('Stamen Toner').add_to(m)
    folium.TileLayer('CartoDB positron').add_to(m)
    folium.TileLayer('CartoDB dark_matter').add_to(m)
    folium.TileLayer(
        tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr='Google Satellite',
        name='Google Satellite',
        overlay=False,
        control=True
    ).add_to(m)
    folium.LayerControl().add_to(m)

    for u, v in zip(route[:-1], route[1:]):
        color = get_edge_color(u, v, 0)
        coords = [(graph.nodes[u]['y'], graph.nodes[u]['x']), (graph.nodes[v]['y'], graph.nodes[v]['x'])]
        folium.PolyLine(coords, color=color, weight=5).add_to(m)

    folium.Marker(route_coords[0], tooltip="Start", icon=folium.Icon(color='green')).add_to(m)
    folium.Marker(route_coords[-1], tooltip="End", icon=folium.Icon(color='red')).add_to(m)

    mid_index = len(route_coords) // 2
    mid_lat, mid_lon = route_coords[mid_index]
    add_pois_to_map(m, mid_lat, mid_lon)

    m.add_child(folium.LatLngPopup())

    with output_map:
        clear_output(wait=True)
        display(m)
    with output_distance:
        clear_output(wait=True)
        print(f"Distance: {route_length/1000:.2f} km")
        print(f"Estimated Time: {total_time:.1f} min")
    with output_elevation:
        clear_output(wait=True)
        plot_elevation(route)

    display(VBox([instruction, legend_html, start_city_widget, end_city_widget, mode_dropdown, button]))
    display(VBox([output_distance, output_map, output_elevation]))

def map_click_callback(lat, lng):
    if clicked_points["start"] is None:
        clicked_points["start"] = (lat, lng)
        print(f"Start point set at ({lat:.5f}, {lng:.5f})")
    elif clicked_points["end"] is None:
        clicked_points["end"] = (lat, lng)
        print(f"End point set at ({lat:.5f}, {lng:.5f})")
    else:
        clicked_points["start"] = (lat, lng)
        clicked_points["end"] = None
        print(f"Start point reset at ({lat:.5f}, {lng:.5f})")
    calculate_route("", "", use_click=True, mode=mode_dropdown.value)

# --- Widgets ---
start_city_widget = Text(description="Start", value="Mumbai")
end_city_widget = Text(description="End", value="Bandra")
mode_dropdown = Dropdown(options=['shortest', 'fastest'], description="Route:")
button = Button(description="Plan Route", button_style='success')

instruction = HTML("""
<h3>🚦 Mumbai Route Planner</h3>
<p>Type locations or click on the map to set start and end points.<br>
1st click: Start | 2nd click: End | 3rd click: Reset Start</p>
""")

legend_html = HTML('''
<div style="padding:10px; border:2px solid #ccc; width:300px; background:white">
<h4>🛳️ Road Type Legend</h4>
<ul style="list-style:none; padding:0">
<li><span style="display:inline-block;width:20px;height:10px;background:red;"></span> Motorway (60 km/h)</li>
<li><span style="display:inline-block;width:20px;height:10px;background:darkred;"></span> Trunk (50 km/h)</li>
<li><span style="display:inline-block;width:20px;height:10px;background:orange;"></span> Primary (40 km/h)</li>
<li><span style="display:inline-block;width:20px;height:10px;background:yellow;"></span> Secondary (35 km/h)</li>
<li><span style="display:inline-block;width:20px;height:10px;background:lightgreen;"></span> Tertiary (30 km/h)</li>
<li><span style="display:inline-block;width:20px;height:10px;background:green;"></span> Residential (25 km/h)</li>
<li><span style="display:inline-block;width:20px;height:10px;background:gray;"></span> Others (20 km/h)</li>
</ul>
</div>
''')

def on_click(btn):
    clicked_points['start'] = None
    clicked_points['end'] = None
    calculate_route(start_city_widget.value, end_city_widget.value, use_click=False, mode=mode_dropdown.value)

start_city_widget.observe(lambda change: on_click(None), names='value')
end_city_widget.observe(lambda change: on_click(None), names='value')
mode_dropdown.observe(lambda change: on_click(None), names='value')
button.on_click(on_click)

# Initial display
on_click(None)
